In [90]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [91]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [92]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [93]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [94]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [95]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])



model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)
predictions_train = model.predict(X)

Evaluation : 

In [96]:
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score
print("f1_score : ",f1_score(y, predictions_train, average='weighted') )

print("Jaccard Score : ",jaccard_score(y, predictions_train))

Maybe our n_estimators count is not enough or more, we define a function for that.

In [113]:
param_test = {
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
print(param_test)

In [114]:
from sklearn.model_selection import GridSearchCV
rfc=RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_test, cv= 5)
CV_rfc.fit(X, y)

In [115]:
CV_rfc.best_params_

In [117]:
model = RandomForestClassifier(n_estimators=500, max_depth=4, random_state=42,criterion="entropy")
model.fit(X, y)
predictions = model.predict(X_test)

In [118]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")